In [1]:
import pandas as pd
import csv
import ast

from tqdm import tqdm
from collections import defaultdict

import spacy
import string
import re

import nltk

In [2]:
df_recipe=pd.read_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/MealRec/recipe.csv")

In [3]:
df_recipe.head()

,Unnamed: 0,recipe_id,recipe_name,review_nums,category,aver_rate,image_url,ingredients,cooking_directions,nutritions,reviews,tags
0,0,6698,Mom's Zucchini Bread,9355,appetizer,4.754207,https://images.media-allrecipes.com/userphotos...,all-purpose flour^salt^baking soda^baking powd...,{'directions': u'Prep\n20 m\nCook\n1 h\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{9568256: {'rating': 5, 'followersCount': 0, '...",north-american;breads;easy;beginner-cook;inexp...
1,1,6709,Zucchini Walnut Bread,58,appetizer,4.478873,http://images.media-allrecipes.com/userphotos/...,chopped walnuts^eggs^white sugar^vegetable oil...,"{'directions': u'Whisk together flour, baking ...","{u'niacin': {u'hasCompleteData': True, u'name'...","{1636096: {'rating': 4, 'followersCount': 0, '...",weeknight;breads;fruit;vegetables;kid-friendly...
2,2,6725,Honey Wheat Bread I,2054,appetizer,4.766846,http://images.media-allrecipes.com/userphotos/...,rapid rise yeast^white sugar^warm water (110 d...,{'directions': u'Prep\n25 m\nCook\n35 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1542146: {'rating': 4, 'followersCount': 56, ...",healthy;breads;dietary;low-cholesterol;healthy...
3,3,6730,Chocolate Chip Orange Zucchini Bread,503,dessert,4.674833,https://images.media-allrecipes.com/userphotos...,eggs^white sugar^vegetable oil^vanilla extract...,"{'directions': u'Sift together flour, baking p...","{u'niacin': {u'hasCompleteData': True, u'name'...","{1978376: {'rating': 5, 'followersCount': 0, '...",for-large-groups;healthy;breads;kid-friendly;d...
4,4,6732,Chocolate Chip Pumpkin Bread,866,dessert,4.718112,https://images.media-allrecipes.com/userphotos...,white sugar^pumpkin puree^vegetable oil^water^...,{'directions': u'Prep\n30 m\nCook\n1 h\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1830917: {'rating': 3, 'followersCount': 0, '...",breads;fall;winter;seasonal;quick-breads;4-hou...


In [5]:
## ['Title','Cuisine','Calories','Carbohydrates','Fats','Proteins','Category',
##  'Typical_serving_size', 'Breakfast','Lunch','Snacks','Dinner']

In [6]:
df={}

In [7]:
df['Title']=df_recipe['recipe_name'].values

In [50]:
remove_punc=['!','"','.','?',':','/','*','~','#','$','%',';','+','=','[',']','<','>','@','`','^','\\','_','{','}','|']
remove_punc_str="".join(remove_punc) 

In [51]:
def clean(s):
  s = s.title() # convert to titlecase: All individual words first letter capitalised
  s=s.strip() # Remove trailing and leading spaces
  s=s.lstrip(string.digits+" "+string.punctuation) # remove leading numbers, spaces and punctuations
  s = s.translate(str.maketrans({key: None for key in remove_punc_str})) # remove other unnecessary punctuations
  if(len(s)>3):
    if(s[:2]=="A " or s[:3]=="A'S" or s[:2]=="A-" or s[:2]=="A)" or s[:2]=="A1" or s[:3]=="(A)"):
      s=s[2:]
  else:
    return ''
  s=s.lstrip(string.digits+" "+string.punctuation) # remove leading numbers, spaces and punctuations
  s=s.rstrip(" "+remove_punc_str)  # remove trailing spaces and punctuations. Not digits as titles can end with <recipe> for 1
  s=re.sub(' +', ' ', s) # remove extra spaces from in between
  return s

In [56]:
df['Title']=[clean(val) for val in df['Title']]

In [8]:
df['Category']=df_recipe['category'].values

In [9]:
df['Typical_serving_size']=[100]*len(df_recipe)

In [10]:
nutri=[ast.literal_eval(v) for v in df_recipe['nutritions'].values]

In [11]:
nutri[0].keys()

dict_keys(['niacin', 'sugars', 'sodium', 'carbohydrates', 'vitaminB6', 'calories', 'thiamin', 'fat', 'folate', 'caloriesFromFat', 'calcium', 'fiber', 'magnesium', 'iron', 'cholesterol', 'protein', 'vitaminA', 'potassium', 'saturatedFat', 'vitaminC'])

In [12]:
df['Calories']=[v['calories']['amount'] for v in nutri]
df['Fats']=[v['fat']['amount'] for v in nutri]
df['Proteins']=[v['protein']['amount'] for v in nutri]
df['Carbohydrates']=[v['carbohydrates']['amount'] for v in nutri]

In [13]:
df_cul=pd.read_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Culinary-DB/rec.csv")

In [14]:
df_cul.head()

,Recipe ID,Title,Source,Cuisine
0,1,5 spice vegetable fried rice,TARLA_DALAL,Indian Subcontinent
1,2,aachar aaloo,TARLA_DALAL,Indian Subcontinent
2,3,aadu lassan keri nu athanu,TARLA_DALAL,Indian Subcontinent
3,4,aaloo kofta,TARLA_DALAL,Indian Subcontinent
4,5,aaloo tamatar subzi,TARLA_DALAL,Indian Subcontinent


In [15]:
titles=df_cul['Title'].values
cuisine=df_cul['Cuisine'].values
cuisines={titles[i]:cuisine[i] for i in range(len(titles))}
cnt=0
for val in df['Title']:
    if val in cuisines:
        cnt+=1

In [16]:
cnt

2630

In [17]:
df_cul['Cuisine'].unique()

array(['Indian Subcontinent', 'Africa', 'China', 'Thailand',
       'South East Asia', 'Middle East', 'Korea', 'Japan',
       'Australia & NZ', 'Mexico', 'Caribbean', 'South America',
       'Misc.: Belgian', 'DACH Countries', 'Greece', 'France',
       'Misc.: Portugal', 'Italy', 'Misc.: Dutch', 'British Isles',
       'Scandinavia', 'Eastern Europe', 'Spain', 'USA', 'Canada',
       'Misc.: Central America'], dtype=object)

In [63]:
tags=df_recipe['tags'].values
nlp = spacy.load('en_core_web_sm')
scores={
    "NORP": 1,
    "ORG": 3,
    "GPE": 0,
    "LOC": 2,
}
titles=df_cul['Title'].values
titles=[clean(val) for val in titles]
cuisine=df_cul['Cuisine'].values
data={titles[i]:cuisine[i] for i in range(len(titles))}

In [64]:
cuisines=[]
for id,val in enumerate(tqdm(tags)):
    if(df['Title'][id] in data):
        cuisines.append(data[df['Title'][id]])
        continue
    doc = nlp(str(val).replace(';',' ').replace('-',' '))
    ents=[]
    for e in doc.ents:
        if (e.label_=="NORP" or e.label_=="LOC" or e.label_=="GPE" or e.label_=="ORG"):
            ents.append([scores[e.label_],e.text])

    ents=sorted(ents)
    if(len(ents)>0):
        cuisines.append(ents[0][1])
    else:
        cuisines.append("")

100%|██████████| 7280/7280 [01:42<00:00, 71.34it/s] 


In [62]:
len([1 for val in cuisines if len(val)!=0])

3648

In [65]:
count=defaultdict(int)
for val in cuisines:
    count[val]+=1

In [66]:
count

defaultdict(int,
            {'north american': 160,
             '': 3250,
             'breads': 1,
             'USA': 781,
             'american': 593,
             'european': 183,
             'Italy': 448,
             'Middle East': 40,
             'South East Asia': 30,
             'Canada': 159,
             'France': 82,
             'Mexico': 372,
             'British Isles': 68,
             'Australia & NZ': 124,
             'english': 12,
             'Misc.: Dutch': 4,
             'southern united states': 25,
             'Greece': 81,
             'Scandinavia': 27,
             'DACH Countries': 38,
             'canadian': 55,
             'Spain': 16,
             'asian': 145,
             'Eastern Europe': 43,
             'Caribbean': 48,
             'french': 8,
             'China': 59,
             'australian': 27,
             'mexican': 39,
             'Japan': 31,
             'african asian': 1,
             'Africa': 39,
             'greek': 4,

### Final

- North America 160+594+1
- United States 781+25+58+4+1+1
- Europe 184+12+38+2+8+4+1+1+1+4+1+9+16
- Canada 159+56
- Italy 446+4
- Australia & NZ 124+27
- Mexico 372+39
- Middle East 39+1
- Eastern Europe 27+43
- Africa 1+39+2+13
- South East Asia 30+2+46
- France 82+8+1
- British Isles 68
- Greece 81+4
- Asia 146+1
- South America 48+16+1+1
- China 58
- Japan 30
- Indian Subcontinent 60
- Korea 13+1

In [67]:
replacements={
    'breads': '',
    'mango':'',
    'cookies': '',
    'rosh': '',
    'winter one': '',
    'USA': 'United States',
    'american': 'North America',
    'english': 'Europe',
    'southern united states': 'United States',
    'north american': 'North America',
    'canadian': 'Canada',
    'australian': 'Australia & NZ',
    'mexican': 'Mexico',
    'lebanese': 'Middle East',
    'african asian': 'Africa',
    'south african': 'Africa',
    'british': 'Europe',
    'scandinavian european dinner party': 'Eastern Europe',
    'Scandinavia': 'Eastern Europe',
    'DACH Countries': 'Europe',
    'french': 'France',
    'greek': 'Greece',
    'united states': 'United States',
    'scottish': 'Europe',
    'Caribbean': 'South America',
    'italian': 'Italy',
    'african': 'Africa',
    'german': 'Europe',
    'irish': 'Europe',
    'asian thai': 'Asia',
    'asian': 'Asia',
    'south west pacific': 'South East Asia',
    'new zealand': 'Australia & NZ',
    'french european': 'France',
    'central american': 'North America',
    'brazilian': 'Europe',
    'european potluck dinner': 'Europe',
    'czech':'Europe',
    'northeastern united states': 'North America',
    'south american': 'South America',
    'danish': 'Europe',
    'superbowl': 'United States',
    'hawaiian': 'United States',
    'iranian': 'Middle East',
    'asian korean': 'Korea',
    'dutch': 'Europe',
    'argentine': 'South America',
    'norwegian': 'Europe',
    'ecuadorean': 'South America',
    'caribbean central american': 'South America',
    'Misc.: Belgian': 'Europe',
    'Misc.: Dutch': 'Europe',
    'Misc.: Portugal': 'Europe',
    'pennsylvania': 'United States',
    'Spain': 'Europe',
    'Thailand': 'South East Asia',
    'melanesia': 'South East Asia',
    'european': 'Europe'
}

In [68]:
cuisines=[replacements[k] if k in replacements else k for k in cuisines]

In [70]:
count=defaultdict(int)
cnt=0
for val in cuisines:
    count[val]+=1
    if val!='':
        cnt+=1
print(cnt)
count

4025


defaultdict(int,
            {'North America': 758,
             '': 3255,
             'United States': 872,
             'Europe': 284,
             'Italy': 452,
             'Middle East': 41,
             'South East Asia': 79,
             'Canada': 214,
             'France': 91,
             'Mexico': 411,
             'British Isles': 68,
             'Australia & NZ': 156,
             'Greece': 85,
             'Eastern Europe': 71,
             'Asia': 146,
             'South America': 75,
             'China': 59,
             'Japan': 31,
             'Africa': 55,
             'Indian Subcontinent': 63,
             'Korea': 14})

In [71]:
df['Cuisine']=cuisines

In [72]:
df_kgl=pd.read_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Kaggle Meal Plan\kaggle_summary.csv")

In [73]:
df_kgl.head()

,Unnamed: 0,Title,Breakfast,Lunch,Snacks,Dinner
0,0,Cherry Poke Cake,0,1,0,1
1,1,Complete Breakfast Smoothie,0,1,0,1
2,2,Jambalaya,0,1,0,1
3,3,Lemon Icebox Pie,0,1,0,1
4,4,Dish Chicken Parmesan,0,0,0,1


In [74]:
titles=set(df_kgl['Title'].values)

In [75]:
cnt=0
for val in df['Title']:
    if val in titles:
        cnt+=1

In [76]:
cnt

6225

In [99]:
titles=df_kgl['Title'].values
meal=df_kgl['Breakfast'].values
data={titles[i]:meal[i] for i in range(len(titles))}
df['Breakfast']=[data[k] if k in data else -1 for k in df['Title']]

meal=df_kgl['Lunch'].values
data={titles[i]:meal[i] for i in range(len(titles))}
df['Lunch']=[data[k] if k in data else -1 for k in df['Title']]

meal=df_kgl['Snacks'].values
data={titles[i]:meal[i] for i in range(len(titles))}
df['Snacks']=[data[k] if k in data else -1 for k in df['Title']]

meal=df_kgl['Dinner'].values
data={titles[i]:meal[i] for i in range(len(titles))}
df['Dinner']=[data[k] if k in data else -1 for k in df['Title']]

In [100]:
df['ID']=[i+1 for i in range(len(df['Title']))]

In [101]:
df.keys()

Index(['ID', 'Title', 'Category', 'Cuisine', 'Calories', 'Fats', 'Proteins',
       'Carbohydrates', 'Breakfast', 'Lunch', 'Snacks', 'Dinner',
       'Typical_serving_size'],
      dtype='object')

In [102]:
df=pd.DataFrame(df)
df=df[['ID', 'Title', 'Category','Cuisine', 'Calories', 'Fats',
       'Proteins', 'Carbohydrates', 'Breakfast', 'Lunch', 'Snacks',
       'Dinner','Typical_serving_size']]

In [104]:
df.sample(5)

,ID,Title,Category,Cuisine,Calories,Fats,Proteins,Carbohydrates,Breakfast,Lunch,Snacks,Dinner,Typical_serving_size
240,241,Chocolate Decadence Cake I,dessert,NaN,424.7165,31.142550,4.968791,38.494020,0,1,0,1,100
1856,1857,Hot Chipped Beef Dip,appetizer,NaN,255.1890,25.310290,4.726992,3.193869,0,0,0,1,100
6508,6509,Light And Easy Greek Potato Salad,appetizer,Greece,105.0407,6.856941,1.202796,10.399120,0,1,0,1,100
856,857,Pumpkin Whoopie Pies,dessert,NaN,424.8207,21.677300,3.410936,55.756290,0,0,1,1,100
4114,4115,Black-Eyed Pea Dip,appetizer,Mexico,123.4825,7.128050,2.929962,12.786940,-1,-1,-1,-1,100


In [105]:
df.to_csv('F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Processed\dishes.csv',index=False)

## Ingredients

In [4]:
df_recipe.head()

,Unnamed: 0,recipe_id,recipe_name,review_nums,category,aver_rate,image_url,ingredients,cooking_directions,nutritions,reviews,tags
0,0,6698,Mom's Zucchini Bread,9355,appetizer,4.754207,https://images.media-allrecipes.com/userphotos...,all-purpose flour^salt^baking soda^baking powd...,{'directions': u'Prep\n20 m\nCook\n1 h\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{9568256: {'rating': 5, 'followersCount': 0, '...",north-american;breads;easy;beginner-cook;inexp...
1,1,6709,Zucchini Walnut Bread,58,appetizer,4.478873,http://images.media-allrecipes.com/userphotos/...,chopped walnuts^eggs^white sugar^vegetable oil...,"{'directions': u'Whisk together flour, baking ...","{u'niacin': {u'hasCompleteData': True, u'name'...","{1636096: {'rating': 4, 'followersCount': 0, '...",weeknight;breads;fruit;vegetables;kid-friendly...
2,2,6725,Honey Wheat Bread I,2054,appetizer,4.766846,http://images.media-allrecipes.com/userphotos/...,rapid rise yeast^white sugar^warm water (110 d...,{'directions': u'Prep\n25 m\nCook\n35 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1542146: {'rating': 4, 'followersCount': 56, ...",healthy;breads;dietary;low-cholesterol;healthy...
3,3,6730,Chocolate Chip Orange Zucchini Bread,503,dessert,4.674833,https://images.media-allrecipes.com/userphotos...,eggs^white sugar^vegetable oil^vanilla extract...,"{'directions': u'Sift together flour, baking p...","{u'niacin': {u'hasCompleteData': True, u'name'...","{1978376: {'rating': 5, 'followersCount': 0, '...",for-large-groups;healthy;breads;kid-friendly;d...
4,4,6732,Chocolate Chip Pumpkin Bread,866,dessert,4.718112,https://images.media-allrecipes.com/userphotos...,white sugar^pumpkin puree^vegetable oil^water^...,{'directions': u'Prep\n30 m\nCook\n1 h\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1830917: {'rating': 3, 'followersCount': 0, '...",breads;fall;winter;seasonal;quick-breads;4-hou...


In [5]:
ings=df_recipe['ingredients'].values

In [6]:
' '.join(ings[0].split('^'))

'all-purpose flour salt baking soda baking powder ground cinnamon eggs vegetable oil white sugar vanilla extract grated zucchini chopped walnuts'

In [8]:
# all_ings=set()
cnt=0
for ing in tqdm(ings):
    for val in ing.split('^'):
        tags=nltk.pos_tag(val.split(' '))

100%|██████████| 7280/7280 [02:19<00:00, 52.12it/s]


In [9]:
cnt

0

In [ ]:
pos_tagged=nltk.pos_tag(nltk.tokenize.word_tokenize("all-purpose flour salt baking soda baking powder ground cinnamon eggs vegetable oil white sugar vanilla extract grated zucchini chopped walnuts"))
print(pos_tagged)

[('all-purpose', 'JJ'), ('flour', 'NN'), ('salt', 'NN'), ('baking', 'VBG'), ('soda', 'NN'), ('baking', 'VBG'), ('powder', 'NN'), ('ground', 'NN'), ('cinnamon', 'NN'), ('eggs', 'NNS'), ('vegetable', 'JJ'), ('oil', 'NN'), ('white', 'JJ'), ('sugar', 'NN'), ('vanilla', 'NN'), ('extract', 'NN'), ('grated', 'VBD'), ('zucchini', 'NNP'), ('chopped', 'VBD'), ('walnuts', 'NNS')]


In [ ]:
pos_tagged=nltk.pos_tag(nltk.tokenize.word_tokenize("eggs rise yeast white sugar warm water (110 degrees F/45 degrees C) fluid ounce) can evaporated milk water melted shortening honey salt wheat flour bread flour butter"))
print(pos_tagged)

[('eggs', 'NNS'), ('rise', 'VBP'), ('yeast', 'RB'), ('white', 'JJ'), ('sugar', 'NN'), ('warm', 'JJ'), ('water', 'NN'), ('(', '('), ('110', 'CD'), ('degrees', 'NNS'), ('F/45', 'NNP'), ('degrees', 'NNS'), ('C', 'NNP'), (')', ')'), ('fluid', 'VBP'), ('ounce', 'NN'), (')', ')'), ('can', 'MD'), ('evaporated', 'VBN'), ('milk', 'NN'), ('water', 'NN'), ('melted', 'VBD'), ('shortening', 'VBG'), ('honey', 'NN'), ('salt', 'NN'), ('wheat', 'NN'), ('flour', 'JJ'), ('bread', 'NN'), ('flour', 'NN'), ('butter', 'NN')]


In [11]:
from nltk.chunk import RegexpParser

In [12]:
ings[2]

'rapid rise yeast^white sugar^warm water (110 degrees F/45 degrees C)^fluid ounce) can evaporated milk^water^melted shortening^honey^salt^wheat flour^bread flour^butter'

In [13]:
id=2
text=' <sep> '.join(ings[id].split('^'))
print(ings[id])
text

rapid rise yeast^white sugar^warm water (110 degrees F/45 degrees C)^fluid ounce) can evaporated milk^water^melted shortening^honey^salt^wheat flour^bread flour^butter


'rapid rise yeast <sep> white sugar <sep> warm water (110 degrees F/45 degrees C) <sep> fluid ounce) can evaporated milk <sep> water <sep> melted shortening <sep> honey <sep> salt <sep> wheat flour <sep> bread flour <sep> butter'

In [173]:
## More can be added later if required as there are 
## too many to be removed manually
restricted={"additional","all-purpose","(optional)"}

In [174]:
def isValid(tag):
    for val in str(tag[0][0].split()):
        if val in restricted:
            return 0
    if (tag[1]!='REQ' or any(ch.isdigit() or ch=='®' for ch in str(tag[0][0]))):
        return 0
    return 1

In [210]:
def prepro(s):
    s=s.strip()
    if s=="fluid ounce) cans":
        return "evaporated milk"
    if s=="warm water (100":
        return "warm water"
    return s.lower()

In [212]:
chunker = RegexpParser(r'''
REQ:
{<JJ|VB>?<NN|NNS|NNP><NN|NNS|NNP>?}
''')
ans=set()
for id in tqdm(range(len(ings))):
    for text in ings[id].split('^'):
        res=chunker.parse(nltk.pos_tag(text.split()))
        sel=""
        cnt=0
        for tag in res.pos():
            if isValid(tag):
                sel+=str(tag[0][0])+" "
                cnt+=1
                if cnt>2:
                    break
            else:
                break
        if sel=="":
            continue
        ans.add(prepro(sel))

  4%|▎         | 272/7280 [00:02<00:52, 132.71it/s]

 33%|███▎      | 2404/7280 [00:19<01:10, 68.80it/s] 

100%|██████████| 7280/7280 [01:05<00:00, 111.32it/s]


In [213]:
len(ans)

3888

In [214]:
res=sorted(list(ans))
# res=["evaporated milk" if k=="fluid ounce) cans" else k for k in res]
# res=["warm water" if k=="warm water (100" else k for k in res]
# res=list(set(res))
# res=[k.lower() for k in res]
# res.remove('')
# res=sorted(res)

In [215]:
len(res)

3888

In [217]:
res[0:10]

['acini di pepe',
 'acorn squash',
 'additional cocktail sauce',
 'additional flour',
 'additional oil',
 'adobo',
 'adobo sauce',
 'agave nectar',
 'agave syrup',
 'alfalfa sprouts']

In [183]:
ans_=set()
for id in tqdm(range(len(ings))):
    for text in ings[id].split('^'):
        ans_.add(text)

100%|██████████| 7280/7280 [00:00<00:00, 282651.26it/s]


In [184]:
len(ans_)

8765

While finding vectors for recipes from ingredients we will just split on space and check if any of the words are in vocab as spaced words are not in w2v vocab. Thus cleaning the ingredients is just loss of info for now. So finally we do not need to do it until we find a better way to get vectors for ingredients or if you can put spaced words
I am leaving the code as now, but will just use the split version to get ingredients.

In [185]:
res_=sorted(list(ans_))

In [186]:
res_[:10]

['',
 '+ 1 tablespoon + 1 teaspoon all-purpose flour',
 '1 1/2 inch thick filet mignon steaks',
 '1 inch pieces fresh asparagus',
 '1% low-fat milk',
 '1% milk',
 '100% pure pumpkin',
 '12x30-inch pieces of parchment paper',
 '15 bean mixture soaked overnight',
 '15 bean soup mix']

Turns out the split ingredients are too bad so I am shifting to the older code.

In [187]:
## ['ID','Name','Calories','Fats','Proteins','Carbohydrates','Cost','Expiry']

In [218]:
df={}
df['ID']=[i+1 for i in range(len(res))]
df['Name']=res
df['Calories']=[0]*len(res)
df['Fats']=[0]*len(res)
df['Carbohydrates']=[0]*len(res)
df['Proteins']=[0]*len(res)
df['Cost']=[0]*len(res)
df['Expiry']=[0]*len(res)

In [219]:
df=pd.DataFrame(df)
df.to_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Processed\ingredients.csv",index=False)

In [220]:
df.head()

,ID,Name,Calories,Fats,Carbohydrates,Proteins,Cost,Expiry
0,1,acini di pepe,0,0,0,0,0,0
1,2,acorn squash,0,0,0,0,0,0
2,3,additional cocktail sauce,0,0,0,0,0,0
3,4,additional flour,0,0,0,0,0,0
4,5,additional oil,0,0,0,0,0,0


In [221]:
len(df)

3888

### Map recipes to ingredients

In [222]:
df_rec=pd.read_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Processed\dishes.csv")

In [223]:
len(df_rec)

7280

In [224]:
df_rec.head()

,ID,Title,Category,Cuisine,Calories,Fats,Proteins,Carbohydrates,Breakfast,Lunch,Snacks,Dinner,Typical_serving_size
0,1,Mom'S Zucchini Bread,appetizer,North America,255.1692,13.140870,3.268513,32.05494,1,0,0,0,100
1,2,Zucchini Walnut Bread,appetizer,NaN,276.0908,13.398670,3.977222,36.54568,1,0,0,0,100
2,3,Honey Wheat Bread I,appetizer,NaN,156.4551,4.770610,4.626510,24.45452,0,1,0,1,100
3,4,Chocolate Chip Orange Zucchini Bread,dessert,NaN,280.3475,15.283680,3.563895,34.25740,1,0,0,0,100
4,5,Chocolate Chip Pumpkin Bread,dessert,NaN,210.1581,9.410045,2.556006,30.47000,1,0,0,0,100


In [225]:
ing_to_id={v:k for k,v in enumerate(res)}

In [227]:
chunker = RegexpParser(r'''
REQ:
{<JJ|VB>?<NN|NNS|NNP><NN|NNS|NNP>?}
''')
all_recs=[]
all_ings=[]
for id in tqdm(range(len(ings))):
    for text in ings[id].split('^'):
        res=chunker.parse(nltk.pos_tag(text.split()))
        sel=""
        cnt=0
        for tag in res.pos():
            if isValid(tag):
                sel+=str(tag[0][0])+" "
                cnt+=1
                if cnt>2:
                    break
            else:
                break
        if sel=="":
            continue
        all_recs.append(id+1)
        all_ings.append(ing_to_id[prepro(sel)])

  4%|▎         | 269/7280 [00:02<00:54, 129.26it/s]

 33%|███▎      | 2420/7280 [00:17<00:31, 153.56it/s]

100%|██████████| 7280/7280 [00:58<00:00, 125.16it/s]


In [228]:
len(all_recs),len(all_ings)

(52845, 52845)

In [229]:
## ['Recipe ID','Ingredient ID']

In [231]:
df={}
df['Recipe ID']=all_recs
df['Ingredient ID']=all_ings

In [234]:
df=pd.DataFrame(df)
df.to_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Processed/rec_ing.csv")

In [235]:
df.head()

,Recipe ID,Ingredient ID
0,1,15
1,1,3051
2,1,1584
3,1,1082
4,1,3715


#### Add Recipe Tags and Category to Dataset

In [13]:
df_rec=pd.read_csv('F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Processed/dishes.csv')

In [14]:
## Updated Cell to check if columns were added
df_rec.head()

,ID,Title,Category,Cuisine,Calories,Fats,Proteins,Carbohydrates,Breakfast,Lunch,Snacks,Dinner,Typical_serving_size,Tags
0,1,Mom'S Zucchini Bread,appetizer,North America,255.1692,13.140870,3.268513,32.05494,1,0,0,0,100,north-american;breads;easy;beginner-cook;inexp...
1,2,Zucchini Walnut Bread,appetizer,NaN,276.0908,13.398670,3.977222,36.54568,1,0,0,0,100,weeknight;breads;fruit;vegetables;kid-friendly...
2,3,Honey Wheat Bread I,appetizer,NaN,156.4551,4.770610,4.626510,24.45452,0,1,0,1,100,healthy;breads;dietary;low-cholesterol;healthy...
3,4,Chocolate Chip Orange Zucchini Bread,dessert,NaN,280.3475,15.283680,3.563895,34.25740,1,0,0,0,100,for-large-groups;healthy;breads;kid-friendly;d...
4,5,Chocolate Chip Pumpkin Bread,dessert,NaN,210.1581,9.410045,2.556006,30.47000,1,0,0,0,100,breads;fall;winter;seasonal;quick-breads;4-hou...


In [8]:
df_rec['Category']=df_recipe['category']
df_rec['Tags']=df_recipe['tags']

In [9]:
df_rec.head()

,ID,Title,Category,Cuisine,Calories,Fats,Proteins,Carbohydrates,Breakfast,Lunch,Snacks,Dinner,Typical_serving_size,Tags
0,1,Mom'S Zucchini Bread,appetizer,North America,255.1692,13.140870,3.268513,32.05494,1,0,0,0,100,north-american;breads;easy;beginner-cook;inexp...
1,2,Zucchini Walnut Bread,appetizer,NaN,276.0908,13.398670,3.977222,36.54568,1,0,0,0,100,weeknight;breads;fruit;vegetables;kid-friendly...
2,3,Honey Wheat Bread I,appetizer,NaN,156.4551,4.770610,4.626510,24.45452,0,1,0,1,100,healthy;breads;dietary;low-cholesterol;healthy...
3,4,Chocolate Chip Orange Zucchini Bread,dessert,NaN,280.3475,15.283680,3.563895,34.25740,1,0,0,0,100,for-large-groups;healthy;breads;kid-friendly;d...
4,5,Chocolate Chip Pumpkin Bread,dessert,NaN,210.1581,9.410045,2.556006,30.47000,1,0,0,0,100,breads;fall;winter;seasonal;quick-breads;4-hou...


In [12]:
df_rec.to_csv('F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Processed\dishes.csv',index=False)